In [9]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs


datadir = "../data/"
figdir = "../figures/"


In [10]:
da_depth = xr.open_dataarray(datadir + "bathymetry_filter50_ease.nc")
da_depth

<xarray.DataArray 'depth' (y: 264, x: 264)> Size: 558kB
[69696 values with dtype=float64]
Coordinates:
    lat      (y, x) float32 279kB ...
    lon      (y, x) float32 279kB ...
  * x        (x) int64 2kB 0 1 2 3 4 5 6 7 8 ... 256 257 258 259 260 261 262 263
  * y        (y) int64 2kB 0 1 2 3 4 5 6 7 8 ... 256 257 258 259 260 261 262 263

In [12]:
depthgradx = da_depth.where(~np.isnan(da_depth)).differentiate("x")/25e3
depthgradx.name = "dhdx"
depthgrady = da_depth.where(~np.isnan(da_depth)).differentiate("y")/25e3
depthgrady.name = "dhdy"

In [13]:
ds_depth = xr.merge([da_depth, depthgradx, depthgrady])
ds_depth

<xarray.Dataset> Size: 2MB
Dimensions:  (y: 264, x: 264)
Coordinates:
    lat      (y, x) float32 279kB 47.33 47.5 47.67 47.84 ... 47.67 47.5 47.33
    lon      (y, x) float32 279kB -45.0 -44.78 -44.56 ... 135.4 135.2 135.0
  * x        (x) int64 2kB 0 1 2 3 4 5 6 7 8 ... 256 257 258 259 260 261 262 263
  * y        (y) int64 2kB 0 1 2 3 4 5 6 7 8 ... 256 257 258 259 260 261 262 263
Data variables:
    depth    (y, x) float64 558kB nan nan nan nan nan ... nan nan nan nan nan
    dhdx     (y, x) float64 558kB nan nan nan nan nan ... nan nan nan nan nan
    dhdy     (y, x) float64 558kB nan nan nan nan nan ... nan nan nan nan nan

In [14]:
ds_depth["slope"] = np.sqrt(ds_depth.dhdx**2 + ds_depth.dhdy**2)
ds_depth

<xarray.Dataset> Size: 3MB
Dimensions:  (y: 264, x: 264)
Coordinates:
    lat      (y, x) float32 279kB 47.33 47.5 47.67 47.84 ... 47.67 47.5 47.33
    lon      (y, x) float32 279kB -45.0 -44.78 -44.56 ... 135.4 135.2 135.0
  * x        (x) int64 2kB 0 1 2 3 4 5 6 7 8 ... 256 257 258 259 260 261 262 263
  * y        (y) int64 2kB 0 1 2 3 4 5 6 7 8 ... 256 257 258 259 260 261 262 263
Data variables:
    depth    (y, x) float64 558kB nan nan nan nan nan ... nan nan nan nan nan
    dhdx     (y, x) float64 558kB nan nan nan nan nan ... nan nan nan nan nan
    dhdy     (y, x) float64 558kB nan nan nan nan nan ... nan nan nan nan nan
    slope    (y, x) float64 558kB nan nan nan nan nan ... nan nan nan nan nan

In [15]:
ds_depth.to_netcdf(datadir + "bathymetry_with_gradients_filter50_ease.nc")